<a href="https://colab.research.google.com/github/tabang205/Khai_Pha_Du_Lieu/blob/main/App_KPDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.4 MB/s eta 0:00:00


In [6]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

# --- 1. CẤU HÌNH TRANG WEB ---
st.set_page_config(
    page_title="Đánh Giá Chất lượng Xe",
    page_icon="🚗",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- 2. LOAD MODEL & DATA ---
@st.cache_resource
def load_model():
    try:
        # Load model đã train
        return joblib.load('car_model_rf.pkl')
    except FileNotFoundError:
        return None

model = load_model()

# --- 3. SIDEBAR - NHẬP LIỆU ---
st.sidebar.title("🔧 Cấu hình xe")
st.sidebar.markdown("---")

def user_input_features():
    # --- MAPPING DATA (Phải khớp với logic lúc train model) ---
    # 1. Buying Price
    price_map = {'Thấp (Low)': 0, 'Trung bình (Med)': 1, 'Cao (High)': 2, 'Rất cao (V-High)': 3}
    buying = st.sidebar.selectbox('1. Phân khúc giá niêm yết', list(price_map.keys()))

    # 2. Maintenance Cost
    maint = st.sidebar.selectbox('2. Chi phí nuôi xe (Bảo dưỡng)', list(price_map.keys()))

    # 3. Doors
    doors_map = {'2': 2, '3': 3, '4': 4, '5 trở lên': 5}
    doors = st.sidebar.selectbox('3. Số lượng cửa', list(doors_map.keys()))

    # 4. Persons
    persons_map = {'2': 2, '4': 4, 'Nhiều hơn (More)': 5}
    persons = st.sidebar.selectbox('4. Số chỗ ngồi', list(persons_map.keys()))

    # 5. Luggage Boot
    boot_map = {'Nhỏ (Small)': 0, 'Vừa (Med)': 1, 'Lớn (Big)': 2}
    lug_boot = st.sidebar.selectbox('5. Kích thước cốp xe', list(boot_map.keys()))

    # 6. Safety
    safe_map = {'Thấp (Low)': 0, 'Trung bình (Med)': 1, 'Cao (High)': 2}
    safety = st.sidebar.selectbox('6. Tiêu chuẩn an toàn', list(safe_map.keys()))

    # --- TẠO DATAFRAME ĐỂ DỰ ĐOÁN ---
    data_input = {
        'buying': price_map[buying],
        'maint': price_map[maint],
        'doors': doors_map[doors],
        'persons': persons_map[persons],
        'lug_boot': boot_map[lug_boot],
        'safety': safe_map[safety]
    }

    # --- TẠO DICT ĐỂ HIỂN THỊ LABEL ---
    display_info = {
        'Giá': buying,
        'Bảo dưỡng': maint,
        'Cửa': doors,
        'Chỗ': persons,
        'Cốp': lug_boot,
        'An toàn': safety
    }

    return pd.DataFrame(data_input, index=[0]), display_info

input_df, display_info = user_input_features()

# --- 4. GIAO DIỆN CHÍNH (TABS) ---
st.title("🚗 Hệ thống Đánh giá & Phân loại Chất lượng Xe")
st.markdown("##### Ứng dụng hỗ trợ ra quyết định mua xe dựa trên học máy (Random Forest)")

# Chia 3 Tab theo đúng yêu cầu
tab1, tab2, tab3 = st.tabs(["🎯 Ứng dụng Dự đoán", "📊 Phân tích Dữ liệu (Real-time)", "⚔️ So sánh Mô hình"])

# ==========================================
# TAB 1: ỨNG DỤNG DỰ ĐOÁN (Product)
# ==========================================
with tab1:
    st.markdown("### Thông số xe bạn đã chọn:")

    # Hiển thị thông số đẹp mắt bằng st.metric
    c1, c2, c3, c4, c5, c6 = st.columns(6)
    c1.metric("Giá", display_info['Giá'].split('(')[0])
    c2.metric("Bảo dưỡng", display_info['Bảo dưỡng'].split('(')[0])
    c3.metric("Số cửa", display_info['Cửa'])
    c4.metric("Chỗ ngồi", display_info['Chỗ'])
    c5.metric("Cốp xe", display_info['Cốp'].split('(')[0])
    c6.metric("An toàn", display_info['An toàn'].split('(')[0])

    st.markdown("---")

    # Nút bấm dự đoán
    predict_btn = st.button("🔍 Thẩm định chất lượng ngay", type="primary", use_container_width=True)

    if predict_btn:
        if model is not None:
            # Thực hiện dự đoán
            prediction = model.predict(input_df)[0]

            # Mapping kết quả từ số sang chữ (Quan trọng)
            # Lưu ý: Class unacc(0), acc(1), good(2), vgood(3)
            result_config = {
                0: {"label": "KHÔNG NÊN MUA (Unacceptable)", "color": "error", "icon": "❌", "msg": "Chất lượng xe thấp hoặc rủi ro an toàn cao."},
                1: {"label": "CHẤP NHẬN ĐƯỢC (Acceptable)", "color": "warning", "icon": "🤔", "msg": "Xe ở mức trung bình, cân nhắc kỹ giá cả."},
                2: {"label": "XE TỐT (Good)", "color": "success", "icon": "✅", "msg": "Chất lượng tốt, đáng đồng tiền bát gạo."},
                3: {"label": "RẤT TỐT (Very Good)", "color": "success", "icon": "💎", "msg": "Lựa chọn tuyệt vời, đầy đủ tiện nghi và an toàn."}
            }

            res = result_config.get(prediction, {"label": "Không xác định", "color": "info"})

            # Hiển thị kết quả
            st.markdown(f"### Kết quả thẩm định: {res['icon']} {res['label']}")

            if res['color'] == 'success':
                st.success(res['msg'])
                st.balloons()
            elif res['color'] == 'warning':
                st.warning(res['msg'])
            else:
                st.error(res['msg'])
        else:
            st.error("⚠️ Lỗi: Không tìm thấy file model 'car_model_rf.pkl'. Vui lòng kiểm tra lại thư mục Colab!")

# ==========================================
# TAB 2: TRỰC QUAN HÓA DỮ LIỆU THẬT (Analytics)
# ==========================================
with tab2:
    st.header("Trực quan hóa Dữ liệu Huấn luyện (Car Evaluation)")

    # Đọc file CSV thật
    csv_file = 'car_evaluation.csv'

    if os.path.exists(csv_file):
        # File gốc không có header nên phải gán names
        col_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
        df = pd.read_csv(csv_file, names=col_names)

        st.success(f"✅ Đã tải thành công dữ liệu: {df.shape[0]} dòng, {df.shape[1]} cột.")

        # Row 1: Pie Chart & Bar Chart
        col_viz1, col_viz2 = st.columns(2)

        with col_viz1:
            st.subheader("1. Tỷ lệ phân lớp (Class Distribution)")
            fig1 = px.pie(df, names='class', color_discrete_sequence=px.colors.sequential.RdBu)
            st.plotly_chart(fig1, use_container_width=True)

        with col_viz2:
            st.subheader("2. Phân bố theo Giá mua")
            fig2 = px.histogram(df, x="buying", color="class", barmode='group',
                                title="Giá mua ảnh hưởng thế nào đến quyết định?",
                                category_orders={"buying": ["low", "med", "high", "vhigh"]})
            st.plotly_chart(fig2, use_container_width=True)

        # Row 2: Sunburst Chart (Phức tạp hơn)
        st.subheader("3. Phân tích đa chiều: An toàn & Sức chứa")
        fig3 = px.sunburst(df, path=['safety', 'persons', 'class'],
                           title="Tương quan: An toàn -> Số chỗ ngồi -> Đánh giá",
                           color='safety',
                           color_discrete_map={'low':'red', 'med':'yellow', 'high':'green'})
        st.plotly_chart(fig3, use_container_width=True)

        with st.expander("Xem dữ liệu gốc (Raw Data)"):
            st.dataframe(df)

    else:
        st.error(f"❌ Không tìm thấy file '{csv_file}'. Bạn hãy upload file này lên Colab để xem biểu đồ nhé!")

# ==========================================
# TAB 3: SO SÁNH MÔ HÌNH (Evaluation)
# ==========================================
with tab3:
    st.header("So sánh Hiệu năng: Random Forest vs Decision Tree")
    st.markdown("Kết quả thực nghiệm trên tập dữ liệu kiểm thử (Test Set).")

    # Dữ liệu so sánh (Lấy từ báo cáo của bạn - nếu có số khác hãy sửa ở đây)
    metrics_data = {
        'Thuật toán': ['Random Forest']*4 + ['Decision Tree']*4,
        'Chỉ số': ['Accuracy', 'Precision', 'Recall', 'F1-Score']*2,
        'Giá trị': [0.97, 0.96, 0.97, 0.96,  # RF Scores (Cao hơn)
                    0.85, 0.82, 0.84, 0.83]   # DT Scores (Thấp hơn)
    }
    df_metrics = pd.DataFrame(metrics_data)

    c_comp1, c_comp2 = st.columns([2, 1])

    with c_comp1:
        st.subheader("4. Biểu đồ so sánh độ chính xác")
        fig4 = px.bar(df_metrics, x="Chỉ số", y="Giá trị", color="Thuật toán", barmode="group",
                      text_auto='.2f', color_discrete_sequence=["#1f77b4", "#ff7f0e"])
        st.plotly_chart(fig4, use_container_width=True)

    with c_comp2:
        st.subheader("5. Kết luận")
        st.info("""
        **Tại sao chọn Random Forest?**

        1. **Độ chính xác cao hơn:** ~97% so với 85%.
        2. **Chống Overfitting:** Nhờ cơ chế Bagging (tập hợp nhiều cây).
        3. **Ổn định:** Ít bị ảnh hưởng bởi nhiễu (noise) trong dữ liệu hơn Decision Tree đơn lẻ.
        """)

    st.markdown("---")
    st.caption("Dự án được thực hiện bởi Bùi Hải Nam, Tạ Văn Bằng - KPDL 2025")

Overwriting app.py


In [3]:
!curl ipv4.icanhazip.com

34.12.227.27


In [8]:
!pip install pyngrok

from pyngrok import ngrok

# Dán Authtoken của bạn vào đây (QUAN TRỌNG)
# Ví dụ: ngrok.set_auth_token("2KvHb...")
ngrok.set_auth_token("36ywysaWvIQxgAgCrq2mk80vf8q_2t4mM7q16d4sQF8bDsUgv")

# hạy Streamlit dưới nền (Background process)
# nohup giúp code chạy ngầm mà không bị treo ô code này
!nohup streamlit run app.py --server.port 8501 &

# Mở kết nối ngrok tới cổng 8501
url = ngrok.connect(8501).public_url
print('🚀 Link App xịn sò của bạn là:', url)

nohup: appending output to 'nohup.out'
🚀 Link App xịn sò của bạn là: https://polymorphistic-unshaking-stacee.ngrok-free.dev


In [7]:
ngrok.kill()